In [ ]:
##NOTE: I RAN A SQL QUERY TO CHANGE currentAge TO age AND currentTeam TO team ON THE PLAYER TABLES!

In [ ]:
#setup dataframe and sql imports
import requests
import pandas as pd
#from urllib.parse import urlencode
#from flatten_json import flatten
import sqlite3

In [ ]:
#setup the base MLB API URL
base_url = "https://statsapi.mlb.com/api"

In [ ]:
#class with defintions for using the API to get data
class MLBStatsAPIClient:
    
    def __init__(self):
        pass
    
    #get game data
    def get_game(self, game_pk, timecode = None, hydrate = None, fields = None):
        request_url = f"{base_url}/v1.1/game/{game_pk}/feed/live"
    
        query_params = {}
    
        if timecode:
            query_params["timecode"] = timecode
            
        if hydrate:
            valid_hydrations = [h for h in hydrate if h in ["credits", "alignment", "flags", "officials", "preState"]]
        
            query_params["hydrate"] = ",".join(valid_hydrations)
        
        if fields:
            query_params["fields"] = ",".join(fields)
        
        response = requests.request(method="GET", url=request_url, params=query_params)
        
        #print(response.request.url)
    
        return pd.DataFrame(response.json())
    
    #get the list of team games by season
    def get_team_schedule_by_season(self, sport_id = None, season = None, game_type = None):
        request_url = f"{base_url}/v1/schedule/"
        
        query_params = {}
            
        if sport_id:
            query_params["sportId"] = sport_id
            
        if season:
            query_params["season"] = season
            
        if game_type:
            query_params["gameType"] = game_type
        
        response = requests.request(method="GET", url=request_url, params=query_params)
        
        #print(response.request.url)
    
        return pd.DataFrame(response.json())
    
    #get the list of teams by season
    def get_team_by_season(self, season = None, sport_id = None):
        request_url = f"{base_url}/v1/teams/"
        
        query_params = {}
        
        if season:
            query_params["season"] = season
            
        if sport_id:
            query_params["sportId"] = sport_id
        
        response = requests.request(method="GET", url=request_url, params=query_params)
        
        #print(response.request.url)
    
        return pd.DataFrame(response.json())
    
    #get list of players by season
    def get_player_by_season(self, sport_id, season = None, game_type = None):
        request_url = f"{base_url}/v1/sports/{sport_id}/players"
        
        query_params = {}
        
        if season:
            query_params["season"] = season
            
        if game_type:
            valid_game_type = [g for g in game_type if g in ["S","R","F","D","L","W","C","N","P","A","I","E"]]
            
            query_params["gameType"] = valid_game_type
            
        response = requests.request(method="GET", url=request_url, params=query_params)
        
        #print(response.request.url)
    
        return pd.DataFrame(response.json())
    
    #get player data
    def get_player(self, person_id, app_context = None, hydrate = None, fields = None):
        request_url = f"{base_url}/v1/people"
        
        query_params = {}
    
        if person_id:
            query_params["personIds"] = person_id
            
        if app_context:
            query_params["appContext"] = app_context
            
        if hydrate:
            #valid_hydrations = [h for h in hydrate if h in ["credits", "alignment", "flags", "officials", "preState"]]
        
            query_params["hydrate"] = ",".join(hydrate)
        
        if fields:
            query_params["fields"] = ",".join(fields)
        
        response = requests.request(method="GET", url=request_url, params=query_params)
        
        #print(response.request.url)
    
        return pd.DataFrame(response.json())

In [ ]:
#initialize the client
client = MLBStatsAPIClient()

In [ ]:
#build a table of teams
def build_teams_table(input_teams_df):
    teams_df = input_teams_df
    team_list = []
    
    teams_df = teams_df.reset_index()
    
    for index, row in teams_df.iterrows():
        team_list.append(row['teams'])
        
    teams_raw = pd.DataFrame(team_list)
    teams_clean = pd.DataFrame()
    
    #only need to grab the id and team name
    teams_clean['id'] = teams_raw['id']
    teams_clean['name'] = teams_raw['name']
    
    #set the id as the index in the dataframe
    teams_clean = teams_clean.set_index(list(teams_clean)[0])
    
    return teams_clean

In [ ]:
#build a table of players
def build_player_table(input_players_df, teams_dict):
    #player dataframe set-up
    players_df = input_players_df
    player_list = []
    player_team = []
    players_df = players_df.reset_index()

    for index, row in players_df.iterrows():
        player_list.append(row['people'])
    
    players_raw = pd.DataFrame(player_list)
    
    #clean out columns we don't need
    players_clean = players_raw.drop(columns=['link','firstName','primaryNumber','birthCity','middleName','boxscoreName',
                                                    'nickName','gender','isVerified','pronunciation','nameFirstLast',
                                                   'nameSlug','firstLastName','lastFirstName','lastInitName',
                                                   'initLastName','fullFMLName','fullLFMName','birthStateProvince',
                                                   'nameMatrilineal','nameTitle','isPlayer'])
    try:
        players_clean = players_clean.drop(columns=['lastPlayedDate'])
    except:
        pass

    players_clean['draftYear'] = players_clean['draftYear'].astype('Int64')
    #players_clean['active'] = players_clean['active'].astype(bool)
    #players_clean['isPlayer'] = players_clean['isPlayer'].astype(bool)
    
    player_team = []
    player_position = []
    player_batside = []
    player_pitchhand = []

    count = 0

    #grab currentTeam, primaryPosition, batSide, pitchHand since embedded
    for count in range(len(players_clean)):
        player_team.append(teams_dict[players_clean['currentTeam'][count]['id']])
        player_position.append(players_clean['primaryPosition'][count]['abbreviation'])
        player_batside.append(players_clean['batSide'][count]['code'])
        player_pitchhand.append(players_clean['pitchHand'][count]['code'])
        
    player_team_clean = pd.DataFrame(player_team)
    players_clean['currentTeam'] = player_team_clean

    player_position_clean = pd.DataFrame(player_position)
    players_clean['primaryPosition'] = player_position_clean

    player_batside_clean = pd.DataFrame(player_batside)
    players_clean['batSide'] = player_batside_clean

    player_pitchhand_clean = pd.DataFrame(player_pitchhand)
    players_clean['pitchHand'] = player_pitchhand_clean
    
    #replace firstName with useName and move it closer to the front
    players_clean.insert(2, "firstName", players_clean['useName'])
    #players_clean.insert(3, "lastName", players_clean['boxscoreName'])
    players_clean = players_clean.drop(columns=['useName'])
    
    players_clean = players_clean.rename(columns={"currentAge": "age", "currentTeam": "team"})
    
    players_clean = players_clean[['id','fullName','firstName','lastName','team','height','weight','age','birthDate',
                                   'birthCountry','batSide','pitchHand','primaryPosition','strikeZoneTop','strikeZoneBottom',
                                  'draftYear','mlbDebutDate','active']]

    players_clean = players_clean.set_index(list(players_clean)[0])

    return players_clean

In [ ]:
#build a table of game events
def build_game_events_table(events, cur_game_id, cur_game_date):
    
    balls = []
    strikes = []
    outs = []
    description = []
    is_strike = []
    is_ball = []
    in_play = []
    pitch_type = []
    pitch_speed = []
    pitch_move_x = []
    pitch_move_z = []
    pitch_loc_x = []
    pitch_loc_z = []
    spin_rate = []
    exit_velocity = []
    launch_angle = []
    hit_distance = []

    batter_id = []
    #batter_name = []
    pitcher_id = []
    #pitcher_name = []
    inning = []
    game_date = []
    game_id = []
    
    #id_of_game = cur_game_id
    #date_of_game = game_date


    atbat = 0
    pitch = 0

    for atbat in range(len(events)):
        for pitch in range(len(events['playEvents'][atbat])):

            #if takes too long to load/too much memory -> https://statsapi.mlb.com/api/v1.1/game/634642/feed/live?fields=liveData,plays,allPlays,playEvents,details,description,isStrike,isBall,isInPlay,type,code

            if events['playEvents'][atbat][pitch]['isPitch']:
                if (events['playEvents'][atbat][pitch]['details']['description']) == "Automatic Ball":
                    continue
                else:
                    description.append(events['playEvents'][atbat][pitch]['details']['description'])
                    is_strike.append(events['playEvents'][atbat][pitch]['details']['isStrike'])
                    is_ball.append(events['playEvents'][atbat][pitch]['details']['isBall'])
                    in_play.append(events['playEvents'][atbat][pitch]['details']['isInPlay'])
                    try:
                        pitch_type.append(events['playEvents'][atbat][pitch]['details']['type']['code'])
                    except:
                        pitch_type.append(None)
                    #print(atbat)
                    #print(pitch)
                    try:
                        pitch_speed.append(events['playEvents'][atbat][pitch]['pitchData']['startSpeed'])
                    except:
                        pitch_speed.append(None)
                    try:
                        pitch_move_x.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pfxX'])
                    except:
                        pitch_move_x.append(None)
                    try:
                        pitch_move_z.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pfxZ'])
                    except:
                        pitch_move_z.append(None)
                    try:
                        pitch_loc_x.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pX'])
                    except:
                        pitch_loc_x.append(None)
                    try:
                        pitch_loc_z.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pZ'])
                    except:
                        pitch_loc_z.append(None)
                    try:
                        spin_rate.append(events['playEvents'][atbat][pitch]['pitchData']['breaks']['spinRate'])
                    except:
                        spin_rate.append(None)
                        
                    balls.append(events['playEvents'][atbat][pitch]['count']['balls'])
                    strikes.append(events['playEvents'][atbat][pitch]['count']['strikes'])
                    outs.append(events['playEvents'][atbat][pitch]['count']['outs'])

                    if events['playEvents'][atbat][pitch]['details']['isInPlay']:
                        try:
                            exit_velocity.append(events['playEvents'][atbat][pitch]['hitData']['launchSpeed'])
                        except:
                            exit_velocity.append(None)
                        try:
                            launch_angle.append(events['playEvents'][atbat][pitch]['hitData']['launchAngle'])
                        except:
                            launch_angle.append(None)
                        try:
                            hit_distance.append(events['playEvents'][atbat][pitch]['hitData']['totalDistance'])
                        except:
                            hit_distance.append(None)
                    else:
                        exit_velocity.append(None)
                        launch_angle.append(None)
                        hit_distance.append(None)

                    inning.append(events['about'][atbat]['inning'])
                    #game_date.append(events['about'][atbat]['startTime'].split("T", 1)[0])
                    batter_id.append(events['matchup'][atbat]['batter']['id'])
                    pitcher_id.append(events['matchup'][atbat]['pitcher']['id'])
                    #batter_name.append(events['matchup'][atbat]['batter']['fullName'])
                    #pitcher_name.append(events['matchup'][atbat]['pitcher']['fullName'])
                    game_id.append(cur_game_id)
                    game_date.append(cur_game_date)
                    #print(game_id)
                    #print(game_date)
                
    #print(pitch_type)

    event_data = pd.DataFrame()

    event_data['gameId'] = game_id
    event_data['pitchType'] = pitch_type
    
    event_data['pitchSpeed'] = pitch_speed
    event_data['pitchSpeed'] = event_data['pitchSpeed'].astype(float)
    
    event_data['description'] = description
    event_data['pitchMovementX'] = pitch_move_x
    event_data['pitchMovementX'] = event_data['pitchMovementX'].astype(float)
    
    event_data['pitchMovementZ'] = pitch_move_z
    event_data['pitchMovementZ'] = event_data['pitchMovementZ'].astype(float)
    
    event_data['pitchLocationX'] = pitch_loc_x
    event_data['pitchLocationX'] = event_data['pitchLocationX'].astype(float)
    
    event_data['pitchLocationZ'] = pitch_loc_z
    event_data['pitchLocationZ'] = event_data['pitchLocationZ'].astype(float)

    event_data['spinRate'] = spin_rate
    event_data['spinRate'] = event_data['spinRate'].astype('Int64')
    
    event_data['exitVelocity'] = exit_velocity
    event_data['exitVelocity'] = event_data['exitVelocity'].astype(float)

    event_data['launchAngle'] = launch_angle
    event_data['launchAngle'] = event_data['launchAngle'].astype(float)

    event_data['hitDistance'] = hit_distance
    event_data['hitDistance'] = event_data['hitDistance'].astype(float)

    event_data['balls'] = balls
    event_data['balls'] = event_data['balls'].astype(int)
    
    event_data['strikes'] = strikes
    event_data['strikes'] = event_data['strikes'].astype(int)
    
    event_data['outs'] = outs
    event_data['outs'] = event_data['outs'].astype(int)
    
    event_data['isStrike'] = is_strike
    event_data['isStrike'] = event_data['isStrike'].astype(bool)
    
    event_data['isBall'] = is_ball
    event_data['isBall'] = event_data['isBall'].astype(bool)

    event_data['isInPlay'] = in_play
    event_data['isInPlay'] = event_data['isInPlay'].astype(bool)

    event_data['batterId'] = batter_id
    event_data['batterId'] = event_data['batterId'].astype(int)

    #event_data['batterName'] = batter_name
    event_data['pitcherId'] = pitcher_id
    event_data['pitcherId'] = event_data['pitcherId'].astype(int)

    #event_data['pitcherName'] = pitcher_name
    event_data['inning'] = inning
    event_data['inning'] = event_data['inning'].astype(int)

    event_data['gameDate'] = game_date

    event_data = event_data.set_index(list(event_data)[0])
    
    return event_data

In [ ]:
#build a dataframe of MLB Teams
teams_2022_df = client.get_team_by_season(2022,1)
teams_table = build_teams_table(teams_2022_df)

In [ ]:
#turn the MLB Teams dataframe into a dictionary (Key: ID, Value: Name)
teams_dict = teams_table.to_dict('dict')
teams_dict = teams_dict['name']

In [ ]:
#build a dataframe of all players from 2021 MLB Regular Season
players_2021_df = client.get_player_by_season(1,2021,"R")
player_table_2021 = build_player_table(players_2021_df,teams_dict)

#save 2021 team to add it to 2022 player list
#player_team_2021 = pd.DataFrame(player_table_2021['currentTeam'])

#add previous year's team into 2022 table
#player_table_2022['2021Team'] = player_team_2021
#player_table_2022.insert(10, "previousTeam", player_table_2022['2021Team'])
#player_table_2022 = player_table_2022.drop(columns=['2021Team'])

In [ ]:
#build a dataframe of all players from 2022 MLB Regular Season
players_2022_df = client.get_player_by_season(1,2022,"R")
player_table_2022 = build_player_table(players_2022_df,teams_dict)

In [ ]:
#setup up our SQLite Connection to create a Player_Data Table
conn = sqlite3.connect('2021_2022_Game_Player_Data.db')

#c = conn.cursor()

player_table_2021.to_sql('Player_Data_2021', conn, if_exists='replace')
player_table_2022.to_sql('Player_Data_2022', conn, if_exists='replace')

#game_events_2021.to_sql('Game_Data_2021', conn, if_exists='replace')
#game_events_2022.to_sql('Game_Data_2022', conn, if_exists='replace')

conn.close()

In [ ]:
#Put all of the players together into one dataframe and clean out duplicates
#player_table = pd.DataFrame()
#player_table = player_table_2022.append(player_table_2021)

#if duplicate ID, keep the most recent one
#player_table = player_table[~player_table.index.duplicated(keep='first')]

In [ ]:
#build a dataframe of all game events from 2021 MLB Regular Season
schedule = client.get_team_schedule_by_season(1,2021,"R")

#game_date = []
#game_ids = []
count = 0

game_events_2021 = pd.DataFrame()

for date in range(len(schedule)):
    #game_date.append(schedule['dates'][date]['date'])
    for game in range(len(schedule['dates'][date]['games'])):
        count = count + 1
        print(count)
        cur_game_date = schedule['dates'][date]['date']
        #print(game_date)
        game_id = schedule['dates'][date]['games'][game]['gamePk']
        game_data = client.get_game(game_id)
        game_data = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
        event_table = build_game_events_table(game_data, game_id, cur_game_date)
        game_events_2021 = game_events_2021.append(event_table)
    

In [ ]:
#build a dataframe of all game events from 2022 MLB Regular Season
schedule = client.get_team_schedule_by_season(1,2022,"R")

#game_date = []
#game_ids = []
count = 0

game_events_2022 = pd.DataFrame()

for date in range(len(schedule)):
    #game_date.append(schedule['dates'][date]['date'])
    for game in range(len(schedule['dates'][date]['games'])):
        count = count + 1
        #print(count)
        cur_game_date = schedule['dates'][date]['date']
        game_id = schedule['dates'][date]['games'][game]['gamePk']
        game_data = client.get_game(game_id)
        game_data = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
        event_table = build_game_events_table(game_data, game_id, cur_game_date)
        game_events_2022 = game_events_2022.append(event_table)

In [ ]:
#setup up our SQLite Connection to create a Player_Data Table
conn = sqlite3.connect('2021_2022_Game_Player_Data.db')

#c = conn.cursor()

player_table_2021.to_sql('Player_Data_2021', conn, if_exists='replace')
player_table_2022.to_sql('Player_Data_2022', conn, if_exists='replace')

game_events_2021.to_sql('Game_Data_2021', conn, if_exists='replace')
game_events_2022.to_sql('Game_Data_2022', conn, if_exists='replace')

conn.close()

In [ ]:
## EXTRA NOTES AND DEBUGGING LOOPS

In [ ]:
game_data = client.get_game(633456)
game_data = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
#event_table = build_game_events_table(game_data, 633457)

pitch_type2 = []
pitch_speed2 = []


for atbat in range(len(game_data)):
    for pitch in range(len(game_data['playEvents'][atbat])):

        #if takes too long to load/too much memory -> https://statsapi.mlb.com/api/v1.1/game/634642/feed/live?fields=liveData,plays,allPlays,playEvents,details,description,isStrike,isBall,isInPlay,type,code

        if game_data['playEvents'][atbat][pitch]['isPitch']:
            if (game_data['playEvents'][atbat][pitch]['details']['description']) == "Automatic Ball":
                continue
            else:
                print(atbat)
                print(pitch)
                #description.append(events['playEvents'][atbat][pitch]['details']['description'])
                #is_strike.append(events['playEvents'][atbat][pitch]['details']['isStrike'])
                #is_ball.append(events['playEvents'][atbat][pitch]['details']['isBall'])
                #in_play.append(events['playEvents'][atbat][pitch]['details']['isInPlay'])
                try:
                    pitch_type2.append(game_data['playEvents'][atbat][pitch]['details']['type']['code'])
                except:
                    pitch_type2.append(None)
                    #print(atbat)
                    #print(pitch)
                pitch_speed2.append(game_data['playEvents'][atbat][pitch]['pitchData']['startSpeed'])
                #pitch_move_x.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pfxX'])
                #pitch_move_z.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pfxZ'])
                #pitch_loc_x.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pX'])
                #pitch_loc_z.append(events['playEvents'][atbat][pitch]['pitchData']['coordinates']['pZ'])
                #spin_rate.append(events['playEvents'][atbat][pitch]['pitchData']['breaks']['spinRate'])
                #balls.append(events['playEvents'][atbat][pitch]['count']['balls'])
                #strikes.append(events['playEvents'][atbat][pitch]['count']['strikes'])
                #outs.append(events['playEvents'][atbat][pitch]['count']['outs'])
                #pitch_type2.append(game_data['playEvents'][atbat][pitch]['details']['type']['code'])
                #inning.append(events['about'][atbat]['inning'])
                #game_date.append(events['about'][atbat]['startTime'].split("T", 1)[0])
                #batter_id.append(events['matchup'][atbat]['batter']['id'])
                #pitcher_id.append(events['matchup'][atbat]['pitcher']['id'])
                #batter_name.append(events['matchup'][atbat]['batter']['fullName'])
                #pitcher_name.append(events['matchup'][atbat]['pitcher']['fullName'])
                #game_id.append(cur_game_id)



In [ ]:
print(game_data['playEvents'][17][1]['pitchData'])
print("\n")
print(game_data['playEvents'][17][2]['pitchData'])
print("\n")
print(game_data['playEvents'][17][1]['pitchData'])

In [ ]:
schedule = client.get_team_schedule_by_season(1,2021,"R")
schedule['dates'][0]['date']
#schedule['dates'][0]['games'][0]['gamePk']

In [ ]:
game_data = client.get_game(634642)
#game_data["gameData"]["players"]
#game_data["gameData"]
#flatten(game_data)

game_data_temp = pd.DataFrame(game_data['liveData']['plays']['allPlays'])
#game_test_2 = pd.DataFrame(game_test['playEvents'][0])
events = pd.DataFrame(game_data_temp)

In [ ]:
player_data = client.get_player(660162,app_context="majorLeague",hydrate=["rosterEntries"])
#player_data = client.get_player(621043,season="2021",app_context="majorLeague",hydrate=["team"])#, fields=["people","fullName"])
#player_data["people"][0]